# Kaggle

https://www.kaggle.com/datasets/mariafrenti/age-prediction

# Imports

In [ ]:
#-- Auto reload:
%load_ext autoreload
%autoreload 2

In [12]:
from matplotlib                          import pyplot
from matplotlib.image                    import imread
from torchvision.datasets                import ImageFolder
from torch.optim.lr_scheduler            import OneCycleLR


import numpy                  as np
import matplotlib.pyplot      as plt
import torchvision.transforms as transforms
import torch.nn               as nn
import torch.nn.functional    as F
import torch.optim            as optim
import torchinfo


import torch
import os
import matplotlib
import torchvision
import torchinfo
import gc 
import os
import time


# Load Data

In [7]:
TRAIN_FOLDER = "/home/amitli/Datasets/Age prediction/20-50/train"
TEST_FOLDER  = "/home/amitli/Datasets/Age prediction/20-50/test"


 #-- ImageNet statistics:
vMean = np.array([0.48501961, 0.45795686, 0.40760392])
vStd  = np.array([0.22899216, 0.224     , 0.225     ])

oTransforms = transforms.Compose([
    #transforms.Resize    (224),
    #transforms.CenterCrop(224),
    transforms.ToTensor  (),
    transforms.Normalize (mean=vMean, std=vStd),
])


batchSize            = 32
oDataSet             = ImageFolder(root=TRAIN_FOLDER, transform=oTransforms)
oTrainSet, oTestSet  = torch.utils.data.random_split(oDataSet, np.round([0.9 * len(oDataSet), 0.1 * len(oDataSet)]).astype(int))

oTrainSet.transform  = oTransforms
oTestSet .transform  = oTransforms

oTrainDL  = torch.utils.data.DataLoader(oTrainSet,   batch_size=batchSize, num_workers=2, persistent_workers=True)
oTestDL   = torch.utils.data.DataLoader(oTestSet,    batch_size=batchSize, num_workers=2, persistent_workers=True)

# EDA

In [ ]:
for i, (x, y) in enumerate(oTrainDL):
    print (x[0].shape)
    break

# Prepare GPU

In [3]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
print(DEVICE)

cuda:0


# Model (Resnet50)

In [17]:
def GetPretrainedModel():
    oModel = torchvision.models.resnet50(pretrained=True)
    #-- freeze weights:
    for mParam in oModel.parameters():
        if False == isinstance(mParam, nn.BatchNorm2d):
            mParam.requires_grad = False
        
    #-- Replace classifier head:
    dIn       = oModel.fc.in_features
    oModel.fc = nn.Sequential(
        nn.Linear(dIn, 512), nn.ReLU(),
        nn.Linear(512, 256), nn.ReLU(),
        nn.Linear(256, 128), nn.ReLU(),
        nn.Linear(128, 31)
    )
    
    return oModel

torchinfo.summary(GetPretrainedModel(), (32, 3, 128, 128))

Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   --                        --
├─Conv2d: 1-1                            [32, 64, 64, 64]          (9,408)
├─BatchNorm2d: 1-2                       [32, 64, 64, 64]          (128)
├─ReLU: 1-3                              [32, 64, 64, 64]          --
├─MaxPool2d: 1-4                         [32, 64, 32, 32]          --
├─Sequential: 1-5                        [32, 256, 32, 32]         --
│    └─Bottleneck: 2-1                   [32, 256, 32, 32]         --
│    │    └─Conv2d: 3-1                  [32, 64, 32, 32]          (4,096)
│    │    └─BatchNorm2d: 3-2             [32, 64, 32, 32]          (128)
│    │    └─ReLU: 3-3                    [32, 64, 32, 32]          --
│    │    └─Conv2d: 3-4                  [32, 64, 32, 32]          (36,864)
│    │    └─BatchNorm2d: 3-5             [32, 64, 32, 32]          (128)
│    │    └─ReLU: 3-6                    [32, 64, 32, 32]   

In [13]:
def Accuracy(mScore, vY):
    vHatY = mScore.detach().argmax(dim=1)
    return (vHatY == vY).float().mean().item()

In [14]:
def Epoch(oModel, oDataDL, Loss, Metric, oOptim=None, oScheduler=None, bTrain=True):

    epochLoss   = 0
    epochMetric = 0
    count       = 0
    nIter       = len(oDataDL)
    vLR         = np.full(nIter, np.nan)
    DEVICE      = next(oModel.parameters()).device #-- CPU\GPU


    oModel.train(bTrain) #-- train or test

    #-- Iterate over the mini-batches:
    for ii, (mX, vY) in enumerate(oDataDL):
        #-- Move to device (CPU\GPU):
        mX = mX.to(DEVICE)
        vY = vY.to(DEVICE)
                
        #-- Forward:
        if bTrain == True:
            #-- Store computational graph:
            mZ   = oModel(mX)                        
            loss = Loss(mZ, vY)           
        else:
            with torch.no_grad():
                #-- Do not store computational graph:
                mZ   = oModel(mX)                
                loss = Loss(mZ, vY)

        #-- Backward:
        if bTrain == True:
            oOptim.zero_grad() #-- set gradients to zeros
            loss.backward()    #-- backward
            oOptim.step()      #-- update parameters
            if oScheduler is not None:
                vLR[ii] = oScheduler.get_last_lr()[0]
                oScheduler.step() #-- update learning rate

        Nb           = vY.shape[0]
        count       += Nb
        epochLoss   += Nb * loss.item()
        epochMetric += Nb * Metric(mZ, vY)
        print(f'\r{"Train" if bTrain else "Val"} - Iteration: {ii:3d} ({nIter}): loss = {loss:2.6f}', end='')

    print('', end='\r')
    epochLoss   /= count
    epochMetric /= count

    return epochLoss, epochMetric, vLR

In [15]:
def TrainClassficationModel(oModel, oTrainData, oValData, Loss, nEpochs, oOptim, oScheduler=None):

    vTrainLoss = np.full(nEpochs, np.nan)
    vTrainAcc  = np.full(nEpochs, np.nan)
    vValLoss   = np.full(nEpochs, np.nan)
    vValAcc    = np.full(nEpochs, np.nan)
    vLR        = np.full(0,       np.nan)
    bestAcc    = 0

    for epoch in range(nEpochs):
        startTime                 = time.time()
        trainLoss, trainAcc, vLRi = Epoch(oModel, oTrainData, Loss, Accuracy, oOptim, oScheduler, bTrain=True ) #-- train
        valLoss,   valAcc,   _    = Epoch(oModel, oValData,   Loss, Accuracy,                     bTrain=False) #-- validate
        epochTime                 = time.time() - startTime

        #-- Display:
        print('Epoch '              f'{epoch    :03d}:',   end='')
        print(' | Train loss: '     f'{trainLoss:6.3f}',   end='')
        print(' | Val loss: '       f'{valLoss  :6.3f}',   end='')
        print(' | Train Accuracy: ' f'{trainAcc :6.3f}',   end='')
        print(' | Val Accuracy: '   f'{valAcc   :6.3f}',   end='')
        print(' | epoch time: '     f'{epochTime:6.3f} |', end='')

        vTrainLoss[epoch] = trainLoss
        vTrainAcc [epoch] = trainAcc
        vValLoss  [epoch] = valLoss
        vValAcc   [epoch] = valAcc
        vLR               = np.concatenate([vLR, vLRi])

        #-- Save best model (early stopping):
        if bestAcc < valAcc:
            bestAcc = valAcc
            try:
                torch.save(oModel.state_dict(), 'BestModel.pt')
            except:
                pass
            print(' <-- Checkpoint!')
        else:
            print('')

    #-- Load best model (early stopping):
    oModel.load_state_dict(torch.load('BestModel.pt'))

    return vTrainLoss, vTrainAcc, vValLoss, vValAcc, vLR

# Train

In [18]:
nEpochs    = 20
nIter      = nEpochs * len(oTrainDL)
Loss       = nn.CrossEntropyLoss()
oModel     = GetPretrainedModel     ().to(DEVICE)
oOptim     = optim.AdamW            (oModel.parameters(), lr=1e-4, betas=(0.9, 0.99), weight_decay=2e-4)
oScheduler = OneCycleLR             (oOptim, max_lr=2e-2, total_steps=nIter)
lHistory   = TrainClassficationModel(oModel, oTrainDL, oTestDL, Loss, nEpochs, oOptim, oScheduler)

Epoch 000: | Train loss:  3.391 | Val loss:  3.368 | Train Accuracy:  0.049 | Val Accuracy:  0.044 | epoch time: 51.535 | <-- Checkpoint!
Epoch 001: | Train loss:  3.375 | Val loss:  3.370 | Train Accuracy:  0.053 | Val Accuracy:  0.048 | epoch time: 51.817 | <-- Checkpoint!
Epoch 002: | Train loss:  3.378 | Val loss:  3.370 | Train Accuracy:  0.050 | Val Accuracy:  0.046 | epoch time: 51.473 |
Epoch 003: | Train loss:  3.377 | Val loss:  3.381 | Train Accuracy:  0.051 | Val Accuracy:  0.050 | epoch time: 51.391 | <-- Checkpoint!
Epoch 004: | Train loss:  3.381 | Val loss:  3.369 | Train Accuracy:  0.051 | Val Accuracy:  0.055 | epoch time: 51.360 | <-- Checkpoint!
Epoch 005: | Train loss:  3.378 | Val loss:  3.369 | Train Accuracy:  0.053 | Val Accuracy:  0.051 | epoch time: 51.678 |
Epoch 006: | Train loss:  3.377 | Val loss:  3.374 | Train Accuracy:  0.052 | Val Accuracy:  0.048 | epoch time: 51.977 |
Epoch 007: | Train loss:  3.372 | Val loss:  3.366 | Train Accuracy:  0.054 | Val 

KeyboardInterrupt: 